# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [32]:
import pandas as pd

---
## 2. Load data

In [33]:
df = pd.read_csv('./client_data.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [34]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,t,0.0,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,f,0.0,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,f,0.0,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [35]:
price_df = pd.read_csv('./price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [36]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


In [39]:
# Create a feature that captures the interaction between consumption and forecast consumption
df['consumption_forecast_interaction'] = df['cons_last_month'] * df['forecast_cons_12m']
print(df['consumption_forecast_interaction'])

0             0.00
1             0.00
2             0.00
3             0.00
4        234464.50
           ...    
14601         0.00
14602    114335.89
14603     34079.81
14604         0.00
14605         0.00
Name: consumption_forecast_interaction, Length: 14606, dtype: float64


In [40]:

# Create a feature that captures the trend in consumption over time
df['consumption_trend'] = (df['cons_last_month'] - df['cons_12m']) / df['cons_12m']
print(df['consumption_trend'])


0             NaN
1       -1.000000
2       -1.000000
3       -1.000000
4       -0.881130
           ...   
14601   -1.000000
14602   -0.974941
14603   -0.902928
14604   -1.000000
14605   -1.000000
Name: consumption_trend, Length: 14606, dtype: float64


In [41]:
# Explore correlation with 'churn'
print(df[['consumption_trend', 'churn']].corr())

                   consumption_trend    churn
consumption_trend            1.00000 -0.00928
churn                       -0.00928  1.00000


In [42]:
# Create a feature that captures the interaction between consumption and forecast consumption
df['consumption_forecast_interaction'] = df['cons_last_month'] * df['forecast_cons_12m']
print(df['consumption_forecast_interaction'])

0             0.00
1             0.00
2             0.00
3             0.00
4        234464.50
           ...    
14601         0.00
14602    114335.89
14603     34079.81
14604         0.00
14605         0.00
Name: consumption_forecast_interaction, Length: 14606, dtype: float64


In [43]:
# Explore correlation with 'churn'
print(df[['consumption_forecast_interaction', 'churn']].corr())

                                  consumption_forecast_interaction     churn
consumption_forecast_interaction                          1.000000 -0.033652
churn                                                    -0.033652  1.000000


In [45]:
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn,consumption_trend,consumption_forecast_interaction
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1,NaN,0.0
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0,-1.00000,0.0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0,-1.00000,0.0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0,-1.00000,0.0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0,-0.88113,234464.5


In [46]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [66]:
df.isnull().sum()

id                                    0
channel_sales                         0
cons_12m                              0
cons_gas_12m                          0
cons_last_month                       0
date_activ                            0
date_end                              0
date_modif_prod                       0
date_renewal                          0
forecast_cons_12m                     0
forecast_cons_year                    0
forecast_discount_energy              0
forecast_meter_rent_12m               0
forecast_price_energy_off_peak        0
forecast_price_energy_peak            0
forecast_price_pow_off_peak           0
has_gas                               0
imp_cons                              0
margin_gross_pow_ele                  0
margin_net_pow_ele                    0
nb_prod_act                           0
net_margin                            0
num_years_antig                       0
origin_up                             0
pow_max                               0


In [78]:
df.dropna(subset=['consumption_trend'], inplace=True)



In [80]:
# Define input features and target variable
X = df[['cons_12m', 'cons_gas_12m', 'cons_last_month', 'forecast_cons_12m', 'forecast_meter_rent_12m', 'imp_cons', 'num_years_antig', 'pow_max',  'consumption_trend', 'consumption_forecast_interaction']]
y = df['churn']

In [81]:

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [82]:

# Create Random Forest Classifier object
rf = RandomForestClassifier(n_estimators=100, random_state=42)

In [83]:
# Train the model on the training set
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [84]:

# Make predictions on the testing set
y_pred = rf.predict(X_test)

In [85]:
# Evaluate the performance of the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      2605
           1       0.67      0.04      0.08       293

    accuracy                           0.90      2898
   macro avg       0.78      0.52      0.51      2898
weighted avg       0.88      0.90      0.86      2898



In [86]:
# Calculate feature importance
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': rf.feature_importances_})
feature_importance.sort_values('Importance', ascending=False, inplace=True)


In [87]:
# Print the feature importance
print('Feature Importance:\n', feature_importance)

Feature Importance:
                             Feature  Importance
0                          cons_12m    0.173419
3                 forecast_cons_12m    0.166822
4           forecast_meter_rent_12m    0.161733
7                           pow_max    0.100816
8                 consumption_trend    0.086754
2                   cons_last_month    0.085959
9  consumption_forecast_interaction    0.079523
5                          imp_cons    0.072994
6                   num_years_antig    0.037812
1                      cons_gas_12m    0.034167


## Explanation of the Result 

The Random Forest model performed well in predicting customers who did not churn (class 0) with a precision score of 0.90 and recall score of 1.00. However, the model underperformed in predicting customers who actually churned (class 1) with a precision score of 0.67 and recall score of 0.04.

I chose to use precision, recall, and F1-score as evaluation metrics because they are useful in evaluating the performance of binary classification models, especially in cases where the two classes are imbalanced. Precision measures the percentage of true positives out of all positive predictions, while recall measures the percentage of true positives out of all actual positives. The F1-score is a combination of precision and recall that provides an overall measure of the model's performance.

Advantages of using a Random Forest for this use case include its ability to handle large datasets with many features, its robustness to noise and outliers, and its ability to handle missing data.
Disadvantages include its tendency to overfit if the number of trees in the forest is too high, and its relatively long training time compared to simpler models like logistic regression.

While the model's performance in predicting class 0 is satisfactory, its performance in predicting class 1 is not. The low recall score for class 1 indicates that the model is not able to capture a significant number of customers who are likely to churn. Therefore, there is room for improvement in the model's performance.

Assuming that the cost of losing a customer is significant, the model's ability to identify customers who are likely to churn can provide significant financial benefits to the client. By offering a discount or incentive to these customers, the client can reduce the likelihood of churn and retain valuable customers. However, the financial benefit of using the model depends on several factors, such as the cost of the incentive, the percentage of customers who accept the offer, and the lifetime value of the retained customers. Further analysis and modeling would be required to estimate these factors and the resulting financial benefit.

## ---The End---